# [Chapter 2. Strings and Text](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html)

Almost every useful program involves some kind of processing, whether it is parsing data or generating output.  
This chapter focuses on common problems involving text manipulation, such as pulling apart strings, searching, substitution, lexing, and parsing.  
Many of these tasks can be easily solved using built-in methods of strings.  
However, more complicated operations might require the use of regular expressions or the creation of a full-fledged parser.  
All of those topics will be covered.  
In addition, a few tricky aspects of [working with Unicode](https://en.wikipedia.org/wiki/Unicode) are addressed.  
Let's do this!

## [Splitting Strings on Any of Multiple Delimiters](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_splitting_strings_on_any_of_multiple_delimiters)

### Problem 

You need to split a string into fields, but the delimiters (and spacing around them) aren't consistent throughout the string.

### Solution

The `split()` method of string objects is really meant for very simple cases, and does not allow for multiple delimiters or account for possible whitespace around the delimiters.  
In cases when you need a bit more flexibility, use the [re.split() method](https://docs.python.org/3/library/re.html#re.split):

In [610]:
import re

line = 'asdf fjdk; afed, fjek,asdf,      foo'
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

Here's some [examples from the documentation](https://docs.python.org/3/library/re.html#re.split):

In [611]:
re.split('\W+', 'Words, words, words.')

['Words', 'words', 'words', '']

In [612]:
re.split('(\W+)', 'Words, words, words.')

['Words', ', ', 'words', ', ', 'words', '.', '']

In [613]:
re.split('\W+', 'Words, words, words.', 1)

['Words', 'words, words.']

In [614]:
re.split('[a-f]+', '0a3B9', flags=re.IGNORECASE)

['0', '3', '9']

### Discussion

The `re.split()` function is useful because you can specify multiple patterns for the separator.  
For example, as shown in the solution, the separator is either a comma (,), semicolon (;), or whitespace followed by any amount of extra whitespace.  
Whenever that pattern is found, the entire match becomes the delimiter between whatever fields lie on either side of the match.  
The result is a list of fields, just as with `str.split()`.

When using `re.split()`, you need to be a bit careful should the regular expression pattern involve a capture group enclosed in parentheses.  
If capture groups are used, then the matched text is also included in the result.  
For example, watch what happens here:

In [615]:
line = 'asdf fjdk; afed, fjek,asdf,      foo'
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

Getting the split characters might be useful in certain contexts.  
For example, maybe you need the split characters later on to reform an output string:

In [616]:
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

In [617]:
values = fields[::2]
values

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

In [618]:
delimiters = fields[1::2] + ['']
delimiters

[' ', ';', ',', ',', ',', '']

In [619]:
# Now re-form the line using the same delimiters and values:
''.join(v+d for v, d in zip(values, delimiters))

'asdf fjdk;afed,fjek,asdf,foo'

In [620]:
# The original line for reference:
line

'asdf fjdk; afed, fjek,asdf,      foo'

If you don't want the separator characters in the result, but still need to use parentheses to group parts of the regular expression pattern, make sure you use a [noncapture group](https://stackoverflow.com/questions/3512471/what-is-a-non-capturing-group-what-does-a-question-mark-followed-by-a-colon), specified as `(?:...)`.

In [621]:
re.split(r'(?:,|;|\s)\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

## [Matching Text at the Start or End of a String](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_matching_text_at_the_start_or_end_of_a_string)

### Problem

You need to check the start or end of a string for specific text patterns, such as filename extensions, URL schemes, and so on.

### Solution

A simple way to check the beginning or end of a string is to use the `str.startswith()` or `str.endswith()` methods.

In [622]:
filename = 'spam.txt'
filename.endswith('.txt')

True

In [623]:
filename.startswith('file:')

False

In [624]:
url = 'http://www.python.org'
url.startswith('http:')

True

If you need to check against multiple choices, simply provide a tuple of possibilities to `startswith()` or `endswith()`:

In [625]:
import os

filenames = os.listdir('.')
filenames

['.ipynb_checkpoints',
 '1_Chapter.ipynb',
 '2_Chapter.ipynb',
 'python_ipsum.txt']

In [626]:
[name for name in filenames if name.endswith(('.ipynb', '.txt'))]

['1_Chapter.ipynb', '2_Chapter.ipynb', 'python_ipsum.txt']

In [627]:
any(name.endswith('.ipynb') for name in filenames)

True

Another example, for your edutainment:

In [628]:
from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http:', 'https:', 'ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()
    f.close()

Oddly, this is one part of Python where a tuple is actually required as input.  
If you happen to have the choices specified in a list or set, just make sure that you convert them to a tuple  (using `tuple()`) first.

In [629]:
choices = ['http:', 'ftp']
url = 'http://www.python.org'

In [630]:
url.startswith(tuple(choices))

True

### Discussion

The `startswith()` and `endswith()` methods provide a very convenient way to perform basic prefix and suffix checking.  
Similar operations can be performed with slices, but are far less elegant.

In [631]:
filename = 'spam.txt'
filename[-4:] == '.txt'

True

In [632]:
url = 'http://www.python.org'
url[:5] == 'http:' or url[:6] == 'https:' or url[:4] == 'ftp:'

True

You might also be inclined to use regular expressions as an alternative.

In [633]:
import re

url = 'http://www.python.org'
re.match('http:|https:|ftp:', url)

<_sre.SRE_Match object; span=(0, 5), match='http:'>

This works, but it is often overkill for simple matching.  
Using the str.startswith() or str.endswith() methods is simpler and much faster.

Last, but not least, the `startswith()` and `endswith()` methods look nice when combined with other operations, such as common data reductions.  
In the next example, this statement checks a directory for the presence of certain kinds of files:

## [Matching Strings Using Shell Wildcard Patterns](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_matching_strings_using_shell_wildcard_patterns)

### Problem

You want to match text using the same wildcard patterns that are commonly used when working with Unix shells (e.g., `*.py, Dat[0-9]*.csv`, and so on).

### Solution

The `fnmatch` module provides two functions -- `fnmatch()` and `fnmatchcase()` -- that can be used to perform such matching.

In [634]:
# Here's an example from the Python 3 documentation:

import fnmatch
import os

for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*.txt'):
        print(file)

python_ipsum.txt


In [635]:
from fnmatch import fnmatch, fnmatchcase

In [636]:
fnmatch('foo.txt', '*.txt')

True

In [637]:
fnmatch('foo.txt', '?oo.txt')

True

In [638]:
fnmatch('Dat45.csv', 'Dat[0-9]*')

True

In [639]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

Normally, `fnmatch()` matches patterns using the same case-sensitivity rules as the system's undelying filesystem (which varies based on the operating system).

In [640]:
# On Mac OSX:
fnmatch('foo.txt', '*.TXT')

False

If this distinction matters, use `fnmatchcase()` instead.  
It matches exactly based on the lower- and upper-case conventions that you supply:

In [641]:
fnmatchcase('foo.txt', '*.TXT')

False

An often overlooked feature of these functions is their potential use with data processing of nonfilename strings.  
For example, suppose that you have a list of street addresses like this:

In [642]:
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]

You could write list comprehensions like this:

In [643]:
from fnmatch import fnmatchcase

In [644]:
[addr for addr in addresses if fnmatchcase(addr, '* ST')]

['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']

In [645]:
[addr for addr in addresses if fnmatchcase(addr, '54[0-9][0-9] *CLARK*')]

['5412 N CLARK ST']

### Discussion

The matching performed by `fnmatch` sits somewhere between the functionality of simple string methods and the full power of regular expressions.  
If you're just trying to provide a simple mechanism for allowing wildcards in data processing operations, it's often a reasonable solution.  
If you're actually trying to write code that matches filenames, use the `glob` module instead.  
See ["Getting A Directory Listing"](http://chimera.labs.oreilly.com/books/1230000000393/ch05.html#dirlisting).

## [Matching and Searching for Text Patterns](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_matching_and_searching_for_text_patterns)

### Problem

You want to match or search text for a specific pattern.

### Solution

If the text you're trying to match is a simple literal, you can often just use the basic string methods, such as `str.find(), str.endswith(), str.startswith(),` or similar.

In [646]:
text = 'yeah, but no, but yeah, but no, but yeah'

In [647]:
# Exact match:
text == 'yeah'

False

In [648]:
# Match at the beginning:
text.startswith('yeah')

True

In [649]:
# Match at the end:
text.endswith('no')

False

In [650]:
# Location of the first occurrence:
text.find('no')

10

For more complicated matching, use regular expressions and the `re` module.  
To illustrate the basic mechanics of using regular expressions, suppose you want to match dates specified as digits, such as `"11/27/2012"`:

In [651]:
text1 = '11/27/2012'
text2 = 'Nov 27, 2012'

In [652]:
import re

# Simple matching.
# \d+ means match one or more digits.
if re.match(r'\d+/\d+/\d+', text1):
    print('yes')
else:
    print('no')

yes


In [653]:
if re.match(r'\d+/\d+/\d+', text2):
    print('yes')
else:
    print('no')

no


If you're going to perform a lot of matches using the same pattern, it usually pays to precompile the regular expression pattern into a pattern object first.

In [654]:
datepat = re.compile(r'\d+/\d+/\d+')
if datepat.match(text1):
    print('yes')
else:
    print('no')

yes


In [655]:
if datepat.match(text2):
    print('yes')
else:
    print('no')

no


The `match()` method always tries to find the match at the start of a string.  
If you want to search text for all occurrences of a pattern, use the `findall()` method instead.

In [656]:
text = 'Today is 10/18/2017. PyCon starts 5/9/2018.'
datepat.findall(text)

['10/18/2017', '5/9/2018']

When defining regular expressions, it is common to introduce capture groups by enclosing parts of the pattern in parentheses.

In [657]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
datepat

re.compile(r'(\d+)/(\d+)/(\d+)', re.UNICODE)

Capture groups often simplify subsequent processing of the matched text because of the contents of each group can be extracted individually.

In [658]:
m = datepat.match('11/27/2012')
m

<_sre.SRE_Match object; span=(0, 10), match='11/27/2012'>

In [659]:
# Extract the contents of each group:
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.group(3))
print(m.groups())

11/27/2012
11
27
2012
('11', '27', '2012')


In [660]:
month, day, year = m.groups()
print(month)
print(day)
print(year)

11
27
2012


In [661]:
# Find all matches.
# Notice the splitting into tuples:
text

'Today is 10/18/2017. PyCon starts 5/9/2018.'

In [662]:
datepat.findall(text)

[('10', '18', '2017'), ('5', '9', '2018')]

In [663]:
for month, day, year in datepat.findall(text):
    print('{}-{}-{}'.format(year, month, day))

2017-10-18
2018-5-9


The `findall()` method searches the text and finds all matches, returning them as a list.  
If you want to find matches iteratively, use the `finditer()` method instead:

In [664]:
for m in datepat.finditer(text):
    print(m.groups())

('10', '18', '2017')
('5', '9', '2018')


### Discussion

A basic tutorial on the theory of regular expressions is beyond the scope of this book.  
However, this recipe illustrates the absolute basics of using the `re` module to match and search for text.  
The essential functionality is first compiling a pattern using `re.compile()` and then using methods such as `match(), findall(),` or `finditer()`.  
When specifying patterns, it is relatively common to use raw strings such as `r'(\d+)/(\d+)/(\d+)'`.  
Such strings leave the backslash character uninterpreted, which can be useful in the context of regular expressions.  
Otherwise, you need to use double backslashes such as `'(\\d+)/(\\d+)/(\\d+)'`.  
Be aware that the `match()` method only checks the beginning of a string.  
It’s possible that it will match things you aren’t expecting.

In [665]:
m = datepat.match('11/27/2012abcdef')
m

<_sre.SRE_Match object; span=(0, 10), match='11/27/2012'>

In [666]:
m.group()

'11/27/2012'

If you want an exact match, make sure the pattern includes the end-marker ($), as follows:

In [667]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)$')
datepat.match('11/27/2012abcdef')

In [668]:
datepat.match('11/27/2012')

<_sre.SRE_Match object; span=(0, 10), match='11/27/2012'>

Last, if you're just doing a simple text matching/searching operation, you can often skip the compilation step and use the module-level functions in the `re` module instead.

In [669]:
re.findall(r'(\d+)/(\d+)/(\d+)', text)

[('10', '18', '2017'), ('5', '9', '2018')]

Be aware, though, that if you’re going to perform a lot of matching or searching, it usually pays to compile the pattern first and use it over and over again.  
The module-level functions keep a cache of recently compiled patterns, so there isn’t a huge performance hit, but you’ll save a few lookups and extra processing by using your own compiled pattern.

## [Searching and Replacing Text](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_searching_and_replacing_text)

### Problem

You want to search for and replace a text pattern in a string.

### Solution

For simple literal patterns, use the `str.replace()` method.

In [670]:
text = 'yeah, but no, but yeah, but no, but yeah'

In [671]:
text.replace('yeah', 'yep')

'yep, but no, but yep, but no, but yep'

For more complicated patterns, use the `sub()` functions/methods in the `re` module.  
To illustrate, suppose you want to rewrite dates of the form "11/27/2012" as "2012-11-27".  
Here's a sample of how to do it:

In [672]:
import re

text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
re.sub(r'(\d+)/(\d+)/(\d+)', r'\3-\1-\2', text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

The first argument to `sub()` is the pattern to match and the second argument is the replacement pattern.  
Backslashed digits such as `\3` refer to capture group numbers in the pattern.

If you're going to perform repeated substitutions of the same pattern, consider compiling it first for better performance.

In [673]:
import re 

datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
datepat.sub(r'\3-\1-\2', text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

For more complicated substitutions, it's possible to specify a substitution callback function instead.

In [674]:
from calendar import month_abbr

def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))

datepat.sub(change_date, text)

'Today is 27 Nov 2012. PyCon starts 13 Mar 2013.'

As input, the argument to the substitution callback is a match object, as returned by `match()` or `find()`.  
Use the `.group()` method to extract specific parts of the match.  
The function should return the replacment text.  
If you want to know how many substitutions were made in addition to getting the replacement text, use `re.subn()` instead.

In [675]:
newtext, n = datepat.subn(r'\3-\1-\2', text)

In [676]:
newtext

'Today is 2012-11-27. PyCon starts 2013-3-13.'

In [677]:
n

2

### Discussion

There isn't much more to regular expression search and replace than the `sub()` method shown.  
The trickiest part is specifying the regular expression pattern -- something that's best left as an exercise for the reader.

## [Searching and Replacing Case-Insensitive Text](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_searching_and_replacing_case_insensitive_text)

### Problem

You need to search for and possibly replace text in a case-insensitive manner.

### Solution

To perform case-insensitive text operations, you need to use the `re` module and supply the `re.IGNORECASE` flag to various operations.

In [678]:
text = 'UPPER PYTHON, lower python, Mixed Python'
re.findall('python', text, flags=re.IGNORECASE)

['PYTHON', 'python', 'Python']

In [679]:
re.sub('python', 'snake', text, flags=re.IGNORECASE)

'UPPER snake, lower snake, Mixed snake'

The last example reveals a limitation that replacing text won't match the case of the matched text.  
If you need to fix this, you might have to use a support function, like so:

In [680]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

Here is an example that uses the function above:

In [681]:
re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE)

'UPPER SNAKE, lower snake, Mixed Snake'

### Discussion

For simple cases, simply providing the `re.IGNORECASE` is enough to perform case-insensitive matching.  
However, be aware that this may not be enough for certain kinds of Unicode matching involving case folding.  
See ["Working with Unicode Characters in Regular Expressions"](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#unicodere) for more details.

## [Specifying a Regular Expression for the Shortest Match](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_specifying_a_regular_expression_for_the_shortest_match)

### Problem

You're trying to match a text pattern using regular expressions, but it is identifying the longest possible matches of a pattern.  
Instead, you would like to change it to find the shortest possible match. 

### Solution

This problem often arises in patterns that try to match text enclosed inside a pair of starting and ending delimiters (like a quoted string).

In [682]:
str_pat = re.compile(r'\"(.*)\"')
text1 = 'Computer says "no."'
str_pat.findall(text1)

['no.']

In [683]:
text2 = 'Computer says "no." Phone says "yes."'
str_pat.findall(text2)

['no." Phone says "yes.']

In this example, the pattern `r'\"(.*)\"'` is attempting to match text enclosed inside quotes.  
However, the `*` operator in a regular expression is greedy, so matching is based on finding the longest possible match. Thus, in the second example involving `text2`, it incorrectly matches the two quoted strings.  
To fix this, add the ? modifier after the `*` operator in the pattern, like this:

In [684]:
str_pat = re.compile(r'\"(.*?)\"')
str_pat.findall(text2)

['no.', 'yes.']

This makes the matching nongreedy, and produces the shortest match instead.

### Discussion

This recipe addresses one of the more common problems encountered when writing regular expressions involving the dot (.) character.  
In a pattern, the dot matches any character except a newline.  
However, if you bracket the dot with starting and ending text (such as a quote), matching will try to find the longest possible match to the pattern.  
This causes multiple occurrences of the starting or ending text to be skipped altogether and included in the results of the longer match.  
Adding the ? right after operators such as `*` or `+` forces the matching algorithm to look for the shortest possible match instead.

## [Writing a Regular Expression for Multiline Patterns](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_writing_a_regular_expression_for_multiline_patterns)

### Problem

You're trying to match a block of text using a regular expression, but you need the match to span multiple lines.

### Solution

This problem typically arises in patterns that use the dot(.) to match any character but forget to account for the fact that it doesn't match newlines.  
For example, suppose you are trying to match C-style comments:

In [685]:
comment = re.compile(r'/\*(.*?)\*/')
text1 = '/* this is a comment */'
comment.findall(text1)

[' this is a comment ']

In [686]:
text2 = '''/* this is a 
              multiline comment */
'''
comment.findall(text2)

[]

To fix the problem, you can add support for newlines.

In [687]:
comment = re.compile(r'/\*((?:.|\n)*?)\*/')
comment.findall(text2)

[' this is a \n              multiline comment ']

In this pattern, `(?:.|\n)` specifies a noncapture group, which basically means that it defines a group for the purposes of matching, but that group is not captured separately or numbered).

### Discussion

The `re.compile()` function accepts a flag, `re.DOTALL`, which is useful here.  
It makes the . in a regular expression match all characters, including newlines.

In [688]:
comment = re.compile(r'/\*(.*?)\*/', re.DOTALL)
comment.findall(text2)

[' this is a \n              multiline comment ']

Using the `re.DOTALL` flag works fine for simple cases, but might be problematic if you’re working with extremely complicated patterns or a mix of separate regular expressions that have been combined together for the purpose of tokenizing, as described in ["Tokenizing Text"](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#tokenizing).  
If given a choice, it’s usually better to define your regular expression pattern so that it works correctly without the need for extra flags.

## [Normalizing Unicode Text to a Standard Representation](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#normalizingunicodetext)

### Problem

You're working with Unicode strings, but you need to make sure that all of the strings have the same underlying representation.

### Solution

In Unicode, certain characters can be represented by more than one valid sequence of code points.  
To illustrate, consider the following example:

In [689]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'

In [690]:
print(len(s1))
s1

14


'Spicy Jalapeño'

In [691]:
print(len(s2))
s2

15


'Spicy Jalapeño'

Here the text "Spicy Jalapeño" has been presented in two forms.  
The first uses the fully composed "ñ" character (U+00F1).  
The second uses the Latin letter "n" followed by a "~" combining character (U+0303).

Having multiple representations is a problem for programs that compare strings.  
In order to fix this, you should first normalize the text into a standard representation using the [unicodedata module](https://docs.python.org/3/library/unicodedata.html):

In [692]:
import unicodedata

t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)

In [693]:
t1 == t2

True

In [694]:
print(ascii(t1))
t1

'Spicy Jalape\xf1o'


'Spicy Jalapeño'

In [695]:
print(ascii(t2))
t2

'Spicy Jalape\xf1o'


'Spicy Jalapeño'

In [696]:
t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)

In [697]:
t3 == t4

True

In [698]:
print(len(t3))
print(ascii(t3))
t3

15
'Spicy Jalapen\u0303o'


'Spicy Jalapeño'

In [699]:
print(len(t4))
print(ascii(t4))
t4

15
'Spicy Jalapen\u0303o'


'Spicy Jalapeño'

The first argument to `normalize()` specifies how you want the string normalized.  
`NFC` means that characters should be fully composed (i.e., use a single code point if possible).  
`NFD` means that characters should be fully decomposed with the use of combining characters.

Python also supports the normalization forms [NFKC and NFKD](https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize), which add extra compatibility features for dealing with certain kinds of characters.

In [700]:
# A single character:
s = '\ufb01'
s

'ﬁ'

In [701]:
unicodedata.normalize('NFD', s)

'ﬁ'

In [702]:
# Notice how the combined letters are broken apart here:
print(unicodedata.normalize('NFKD', s))
print(unicodedata.normalize('NFKC', s))

fi
fi


### Discussion

Normalization is an important part of any code that needs to ensure that it processes Unicode text in a sane and consistent way.  
This is especially true when processing strings received as part of user input where you have little control over the encoding.

Normalization can also be an important part of sanitizing and filtering text.  
For example, suppose you want to remove all diacritical marks from some text (possibly for the purposes of searching or matching):

In [703]:
t1 = unicodedata.normalize('NFD', s1)
''.join(c for c in t1 if not unicodedata.combining(c))

'Spicy Jalapeno'

This last example shows another important aspect of the `unicodedata module` -- namely, utility functions for testing characters against character classes.  
The `combining()` function tests a character to see if it is a combining character.  
There are other functions in the module for finding character categories, testing digits, and so forth.

Unicode is obviously a large topic.  
For more detailed reference information about normalization, visit [Unicode’s page on the subject](https://www.unicode.org/faq/normalization.html).  
Ned Batchelder has also given an excellent presentation on Python Unicode handling issues [on his website](https://nedbatchelder.com/text/unipain.html).

## [Working with Unicode Characters in Regular Expressions](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#unicodere)

### Problem

You are using regular expressions to process text, but are concerned about the handling of Unicode characters.

### Solution

By default, the `re` module is already programmed with rudimentary knowledge of certain Unicode character classes.  
For example, `\d` already matches any unicode digit character:

In [704]:
import re

num = re.compile('\d+')
num

re.compile(r'\d+', re.UNICODE)

In [705]:
# ASCII digits:
num.match('123')

<_sre.SRE_Match object; span=(0, 3), match='123'>

In [706]:
# Arabic digits:
num.match('\u0661\u0662\u0663')

<_sre.SRE_Match object; span=(0, 3), match='١٢٣'>

If you need to include specific Unicode characters in patterns, you can use the usual escape sequence for Unicode characters (e.g., `\uFFFF` or `\UFFFFFFF`).  
For example, here is a regex that matches all characters in a few different Arabic code pages:

In [707]:
arabic = re.compile('[\u0600-\u06ff\u0750-\u077f\u08a0-\u08ff]+')
arabic

re.compile(r'[\u0600-ۿݐ-ݿࢠ-ࣿ]+', re.UNICODE)

When performing matching and searching operations, it’s a good idea to normalize and possibly sanitize all text to a standard form first (see ["Normalizing Unicode Text to a Standard Representation"](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#normalizingunicodetext)).  
However, it’s also important to be aware of special cases.  
For example, consider the behavior of case-insensitive matching combined with case folding:

In [708]:
pat = re.compile('stra\u00dfe', re.IGNORECASE)
pat

re.compile(r'straße', re.IGNORECASE|re.UNICODE)

In [709]:
s = 'straße'
print(pat.match(s))  # Matches

<_sre.SRE_Match object; span=(0, 6), match='straße'>


In [710]:
print(pat.match(s.upper()))  # Doesn't match

None


In [711]:
s.upper()  # Case folds

'STRASSE'

### Discussion

Mixing Unicode and regular expressions is often a good way to make your head explode.  
If you’re going to do it seriously, you should consider installing the [third-party regex library](https://pypi.python.org/pypi/regex), which provides full support for Unicode case folding, as well as a variety of other interesting features, including approximate matching.

## [Stripping Unwanted Characters from Strings](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_stripping_unwanted_characters_from_strings)

### Problem

You want to strip unwanted characters, such as whitespace, from the beginning, end, or middle of a text string.

### Solution

The `strip()` method can be used to strip characters from the beginning or end of a string.  
The `lstrip()` and `rstrip()` methods perform stripping from the left or right side.  
By default, these methods strip whitespace, but other characters can be given.

In [712]:
# Whitespace stripping:
s = '    hello world \n'
s

'    hello world \n'

In [713]:
s.strip()

'hello world'

In [714]:
s.lstrip()

'hello world \n'

In [715]:
s.rstrip()

'    hello world'

In [716]:
# Character stripping:
t = '-----hello====='
t

'-----hello====='

In [717]:
t.lstrip()

'-----hello====='

In [718]:
t.lstrip('-')

'hello====='

In [719]:
t.strip('-')

'hello====='

In [720]:
t.strip('-=')

'hello'

### Discussion

The various `strip()` methods are commonly used when reading and cleaning up data for later processing.  
For example, you can use them to get rid of whitespace, remove quotations, and other tasks.

Be aware that stripping does not apply to any text in the midle of a string.

In [721]:
s = '  hello       world   \n'
s

'  hello       world   \n'

In [722]:
s = s.strip()
s

'hello       world'

If you need to do something to the inner space, you need to use another technique, such as using the `replace()` method or a regular expression substitution.

In [723]:
s.replace(' ', '')

'helloworld'

In [724]:
import re
re.sub('\s+', ' ', s)

'hello world'

It is often the case that you want to combine string stripping operations with some other kind of iterative processing, such as reading lines of data from a file.  
If so, this is one area where a generator expression can be useful:

Here, the expression `lines = (line.strip() for line in f)` acts as a kind of data transform.  
It’s efficient because it doesn’t actually read the data into any kind of temporary list first.  
It just creates an iterator where all of the lines produced have the stripping operation applied to them.

For even more advanced stripping, you might turn to the `translate()` method.  
See the next recipe on sanitizing strings for further details.

## [Sanitizing and Cleaning Up Text](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_sanitizing_and_cleaning_up_text)